# Global Drug Trafficking Networks: Data Preparation

## Author: Matei Gabriel Cosa

## Supervisors: prof. Daniele Durante, prof. Omiros Papaspiliopoulos

### Bocconi University & Bocconi Institute for Data Science and Analytics (BIDSA)

## 1. Introduction

Transnational drug trafficking is a complex and ever-changing phenomenon that is responsible for billions of illicit dollars and, unfortunately, countless lives. Understanding the flows of drugs, the construction of routes, and the roles of different countries requires a rigorous mathematical and computational approach. Applying network-scientific methods and deep learning appears to hold great potential. But to be able to conduct such an analysis, one must first obtain the necessary data and create the appropriate infrastructure for the construction of the network. This is precisely the purpose of this notebook.

## 2. Methodology

### 2.1. Relevant literature 

The main point of reference is Giommoni, L., Berlusconi, G. & Aziani, A. Interdicting International Drug Trafficking: a Network Approach for Coordinated and Targeted Interventions. Eur J Crim Policy Res 28, 545–572 (2022). https://doi.org/10.1007/s10610-020-09473-0. In this paper, the methodology for constructing cocaine and heroin networks is described in detail. We will follow a similar approach.

Additionaly, UNODC reports are used for certain technical adjustments, such as drug conversion rates for different measurement units and purities. The two most important sources are:
- UNODC. (2010). Methodology–World drug report 2010. Vienna: UNODC https://www.unodc. org/documents/data-and-analysis/WDR2010/WDR2010methodology.pdf
- UNODC. (2019b). World drug report 2019 methodology report. Vienna: United Nations Office on Drugs and Crime https://wdr.unodc.org/wdr2019/prelaunch/WDR-2019-Methodology-FINAL.pdf

### 2.2. Data sources

We will be using several different datasets:

1. **World population by country for ages 15-64**: UNDESA "Population by 5-year age groups and sex";
    - accessed through the API, details available at: https://population.un.org/dataportal/about/dataapi
    
    
2. **Drug prevalence by country**: UNODC "Prevalence of drug use in the general population - national data";
    - accessed in .xlsx format from: https://wdr.unodc.org/wdr2019/en/maps-and-tables.html
    
    
3. **Average drug purity by country**: UNODC "Prices and Purities of Drugs";
    - accessed in .xlsx format from: https://www.unodc.org/unodc/en/data-and-analysis/wdr2022_annex.html
    
    
4. **Drug seizures**: UNODC "Individual Drug Seizures" (IDS);
    - accessed in .xlsx format from: https://dmp.unodc.org/
    - IMPORTANT: currently, the data published on the UNODC website are incomplete, because they do not contain           information about routes. The version od the dataset used for this study was kindly provided by prof. Luca             Giommoni from Cardiff University.

5. **Drug prices**: UNODC "Durg Prices"
    - accessed in .xlsx format from: https://dataunodc.un.org/dp-drug-prices
    - we considered drug prices in USD/kg at wholesale level

6. **GDP/capita**: The World Bank "GDP per capita (current US$)"
    - accessed in .xlsx format from: https://data.worldbank.org/indicator/NY.GDP.PCAP.CD

7. **Geographic coordinates**: Kaggle "Countries geographic ccoordinates"
    - accessed in .csv format from: https://www.kaggle.com/datasets/eidanch/counties-geographic-coordinates

8. **Governance indicators**: The World Bank "The Worldwide Governance Indicators (WGI)"
    - accessed in .xlsx format from: https://www.worldbank.org/en/publication/worldwide-governance-indicators
    

### 2.3. Network construction

We are interested in builduing networks of type $N_{d, t}=(I_{d,t}, F_{d,t})$, where $I_{d,t}=\{i_{d,t,1}, i_{d,t,2}, \dots ,i_{d,t,n}\}$ is the set of nodes and $F_{d,t}=\{(j, k)\}$ is the set of weighted edges. Such network $N_{d, t}$ is representative of the drug $d$ in the year $t$. To build such a network, several steps are required:
1. **Identifying the connections**: this is done using data from the *IDS dataset*. Information about *COUNTRY_OF_SEIZURE*, *DEPARTURE_COUNTRY*, and *DESTINATION_COUNTRY* yields pairs of connections, i.e. directed edges.


2. **Computing relative weights of the connections**: this is done using data from the *IDS dataset*. Total flows along each connections are adjusted by the average purity at wholesale level in the *COUNTRY_OF_SEIZURE*. The relevance of each edge is computed as:

$$
F_{d,t,j\leftarrow k}=\frac{S_{d,t,j\leftarrow k}}{\sum_{k=1}^{K} S_{d,t,j\leftarrow k}}
$$

where $S_{d,t,j\leftarrow k}$ is the total quantity of drug $d$ seized in year $t$ that was imported by country $j$ from country $k$.

3. **Estimating the total purity-adjusted quantity of drugs seized and consumed in each country**: this is done using data from all four datasets. Firstly, data on the average national purity of each drug is collected: $pu_{d,t,i}$ represents the average purity of drug $d$, at time $t$, in country $i$. Later, data on the average national prevalence of each drug is collected: $pr_{d,t,i}$ represents the average purity of drug $d$, at time $t$, in country $i$. Then, population data for each country for the age category 15-64 is gathered: $Pop_{t,i}$ represents the total population for the age group at time $t$ and in country $i$. Afterwards, average global consumption per user of drug $d$ at time $t$, $c_{d,t}$ is estimated by dividing the total amount of drug $d$ produced at time $t$ by the total number of users. Additionally, total quantity of drug $d$ seized at time $t$ in country $i$, $S_{d,t,i}$, is computed. This information is later used for estimating the total market for drug $d$ at time $t$ in country $i$, $M_{d,t,i}$ by combining together consumption and seizures:

$$
M_{d,t,i}=C_{d,t,i}^{adj}+S_{d,t,i}^{adj} \\
C_{d,t,i}^{adj}=pr_{d,t,i} \cdot Pop_{t,i} \cdot c_{d,t} \\
S_{d,t,i}^{adj}=S_{d,t,i} \cdot pu_{d,t,i}
$$

4. **Estimating import and export flows**: this is done by combining all the previous data. The idea is that total imports must satisfy the internal market (i.e, consumption & seizures), as well as exports. Imports $I_{d,t,i\leftarrow k}$ and exports $E_{d,t,i\rightarrow l}$ are computed by solving the system:

$$
\begin{cases} 
\sum_{k=1}^{K} I_{d,t,i\leftarrow k} = C_{d,t,i}^{adj}+S_{d,t,i}^{adj} + \sum_{l=1}^{L} E_{d,t,i\rightarrow l} \\ 
\sum_{l=1}^{L} E_{d,t,i\rightarrow l} = \sum_{l=1}^{L} I_{d,t,l\leftarrow i}
\end{cases}
$$

### 2.4. Implementation details

In what follows we shall use the theoretical guidelines of *Giommoni, Berlusconi & Aziani* to as large as an extent as possible without having access to their code. This being said, there are several implementaion choices that we made in order to better suit our goals:
1. **Separate modules**: to keep this notebook clean and readable, we opted for creating separate Python modules to tackle specifc parts that required extensive code.
2. **Drug types**: although the original paper focuses on cocaine on heroin, we attempt to include other drugs, such as cannabis, amphetamine, and ecstasy. We included derivative versions of these drugs to work with more data and paint a brader picture. Our choices might not be entirely scientifically-accurate (e.g., combining amphetamine and methamphetamine). This being said, the purpose of this study is mainly mathematical and computational, therefore we believe our effort in following specific methods present in the literature should be satisfactory.
3. **File types**: for convenience, most of the data used and obtained through our processing is stored in *.xlsx* format. 
4. **Period**: we will look at data from 2006 to 2017.

### 2.5. Preliminaries

For convenience, we shall import useful packages and prepare data that will be useful later on, before jumping into the other steps. An important element consists of obtaining the list of countries present in the *IDS* dataset, so that we can make sure that we collect data (purity, prevalence, etc.) for all countries.

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Import useful packages
import numpy as np
import pandas as pd
import networkx as nx

# Import the our modules
import Population
import Prevalence
import Purity
import Seizures
import Aggregation
import Features

In [3]:
# Read the IDS dataset
df_ids = Seizures.read_xlsx()

In [15]:
# Obtain all countries, sub-regions and regions in the IDS dataset
cocaine_countries_list, cocaine_sub_region_dict, cocaine_region_dict = Seizures.get_ids_locations(df_ids, drug_list = ['Cocaine'])
heroin_countries_list, heroin_sub_region_dict, heroin_region_dict = Seizures.get_ids_locations(df_ids, drug_list = ['Heroin'])
all_countries_list, all_sub_region_dict, all_region_dict = Seizures.get_ids_locations(df_ids, drug_list = ['Cocaine', 'Heroin'])


## 3. Population

We now gather data on total population aged between 15 and 64 from UNDESA. To do this, we used a pre-written module that leverages the UNDESA API. 

*IMPORTANT*: takes long to run; do not re-run multiple times!

In [ ]:
# Get the population data stored in a dict of pd.DataFrames
#population_data = Population.get_population()

# Write the population data to an Excel file, where each year represents a different spreadsheet
#Population.write_to_xlsx(population_data)

## 4. Prevalence

We now gather data on prevalence of drug for population aged between 15 and 64 from UNODC. To do this, we used a pre-written module that leverages data from the UNODC website. Missing data are imputed with sub-regional, regional, or global averages. 

In [3]:
# Prepare data
df_prev = Prevalence.prepare_data()

# Store the prevalence data in output_df
# The missing locations are imputed, ensuring that the final purity dataset contains all countries present in the IDS dataset
output_df = Prevalence.get_prevalence_values(df_prev, countries_list = all_countries_list, sub_regions_dict = all_sub_region_dict, regions_dict = all_region_dict)

# Write the prevalence data to an Excel file, where each year represents a different spreadsheet
Prevalence.write_to_xlsx(output_df)

## 5. Purity

We now gather data on average drug purity at wholesale level from UNODC. To do this, we used a pre-written module that leverages data from the UNODC website. Missing data are imputed with sub-regional, regional, or global averages, as well as linear interpolation. 

In [4]:
# Prepare the purity data
df_pure = Purity.prepare_data()

# Store the prevalence data in output_df
# The missing locations are imputed, ensuring that the final purity dataset contains all countries present in the IDS dataset
output_df = Purity.get_purity_values(df_pure, locations = all_countries_list, sub_region_dict = all_sub_region_dict, region_dict = all_region_dict)

# Write the purity data to an Excel file, where each year represents a different spreadsheet
Purity.write_to_xlsx(output_df)

## 6. Seizures

### 6.1. Outline

The seizures dataset (*IDS*) is by far the larget and most important data source of our project. The goal at this point is two-fold: computing total seized quantities at a national level and retrieving the network structure by extracting edges and weights. This will allow us to obtain all the ingredients we need for estimating imports and exports, thereby completing the network. 

An important note about purity levels is in order. Using guidelines from the UNODC, as well as methods from the literature, we built a conversion module (i.e., *Quantity_Conversion.py*) that automatically translates the seized quantities into *kilograms*. We then multiply this quanities by the average purity levels retrieved earlier. Therefore, all quantities resulting from this step of our processing represent **100% pure kilograms** of illicit substance.

### 6.2. National seizures

We first compute the total quanity of drug seized in each country every year, adjusted for purity and translated to kilograms. For details on the implementation, check https://github.com/MateiCosa/Global-Drug-Network-Analysis/tree/main.

In [11]:
# Get the purity-adjusted national seizures
output_df = Seizures.get_purity_adjusted_seizures(df_ids)

# Write the seizures data to an Excel file, where each year represents a different spreadsheet
Seizures.write_to_xlsx(output_df)

### 6.3. Network edges and weights

Using the *IDS dataset* we extract the all the connections between countries, the total quantity seized across each connection (*weight*), the relative weight of each connection (*relative_weight*), and the status of producing country for each node (*producer*). 

Using the *get_drug_network_by_year* function the *Seizures* package, we create a **dictionary of networks**, where each year is mapped to a *nx.DiGraph* object representing the network of the given year. Each drug network corresponds to one type of drug, so the function must be called for each drug of interest for the analysis.

In [8]:
cocaine_network = Seizures.get_drug_network_by_year('Cocaine', df_ids)
heroin_network = Seizures.get_drug_network_by_year('Heroin', df_ids)

Here some brief summary statistics for the cocaine network:

In [9]:
stats = pd.DataFrame(columns = ['Year', 'Countries', 'Connections'])
for year in range(2006, 2018):
    stats.loc[len(stats)] = {'Year': year, 'Countries': len(cocaine_network[year].nodes), 'Connections': len(cocaine_network[year].edges)}
stats

,Year,Countries,Connections
0,2006,99,163
1,2007,93,154
2,2008,92,145
3,2009,113,250
4,2010,117,256
5,2011,104,235
6,2012,114,346
7,2013,104,271
8,2014,106,241
9,2015,74,145


## 7. Data aggregation

### 7.1. Outline

Now that we have collected and pre-processed data on population, prevalence, purities, and seizures, we need to combine everything together in order to construct the final network. There are several steps that need to pe performed, including estimating the number of users and the average consumption for a given drug, the national market, and, finally, the actual flows between any two countries present in the network.

### 7.2. Estimating the number of drug users

To estimate the number of drug users in every country we simply need to multiply the total population aged between 15 and 64 years old with the prevalence of the drug of interest.

In [159]:
cocaine_users = Aggregation.get_drug_users('Cocaine', cocaine_countries_list)
heroin_users = Aggregation.get_drug_users('Heroin', heroin_countries_list)

Let's preview the output:

In [161]:
cocaine_users[2006]

{'Afghanistan': 71346.05839285716,
 'Albania': 15306.544500000004,
 'Algeria': 4457.410400000001,
 'Andorra': 442.9014857142857,
 'Angola': 2081.1675,
 'Anguilla': 48.733071428571435,
 'Antigua and Barbuda': 316.2384642857143,
 'Argentina': 271695.4056,
 'Armenia': 11802.272785714287,
 'Aruba': 388.41525000000007,
 'Australia': 79821.05946428572,
 'Austria': 42139.40091428572,
 'Azerbaijan': 34520.06796428572,
 'Bahamas': 1760.7825,
 'Bahrain': 4174.737107142858,
 'Bangladesh': 502971.8866071429,
 'Barbados': 736.844,
 'Belarus': 52027.39645714286,
 'Belgium': 52552.95257142857,
 'Benin': 882.9127000000001,
 'Bermuda': 257.70439285714286,
 'Bolivia, Plurinational State of': 61196.2624,
 'Bosnia and Herzegovina': 21367.650857142857,
 'Botswana': 6776.639357142858,
 'Brazil': 1380283.75875,
 'Brunei Darussalam': 1473.942535714286,
 'Bulgaria': 40231.31451428572,
 'Burkina Faso': 1484.8633,
 'Burundi': 23141.101178571433,
 'Cambodia': 48251.52932142858,
 'Cameroon': 54091.70742857143,
 'C

### 7.2. Estimated yearly consumption per user

Since data about consumption per user is unavailable in most countries, a *supply-side* estimation approach is chosen. This requires retrieving estimates of the total quantities of drugs produced every year and dividing them by the total number of global users. There are, of course, several limitations of this method, such as not differentiating between "light" and "heavy" users or ignoring differences in consumption across countries.

In [ ]:
cocaine_consumption = Aggregation.get_yearly_consumption('Cocaine', cocaine_users)
heroin_consumption = Aggregation.get_yearly_consumption('Heroin', cocaine_users)

Let's preview the output:

In [435]:
cocaine_consumption

{2006: 0.0053242324030680944,
 2007: 0.009754496006354864,
 2008: 0.007532680690225996,
 2009: 0.003965445076188423,
 2010: 0.006756212442493026,
 2011: 0.0051178087328603095,
 2012: 0.005337210318074906,
 2013: 0.0017755006907820994,
 2014: 0.0016918860089100132,
 2015: 0.004830003310930263,
 2016: 0.0053035891786425565,
 2017: 0.006672715656815675}

### 7.3. National market estimation

To be able to estimate bilateral flows, we need to have an estimate of the national markets, meaning the sum between total estimated consumption and seizures. It is therefore necessary to merge data from the seizures dataset with data on population, prevalence, and average consumption per user.

$$
M_{d,t,i}=C_{d,t,i}^{adj}+S_{d,t,i}^{adj} \\
C_{d,t,i}^{adj}=pr_{d,t,i} \cdot Pop_{t,i} \cdot c_{d,t} \\
S_{d,t,i}^{adj}=S_{d,t,i} \cdot pu_{d,t,i}
$$

In [16]:
import Aggregation

# Only works for cocaine & heroin since we do not have average consumption data for other drugs
df_markets = Aggregation.get_national_markets_df(all_countries_list, all_sub_region_dict, all_region_dict,
                                                 df_ids, drug_list = ['Cocaine', 'Heroin'])

# Write to file
Aggregation.write_to_xlsx(df_markets)

### 7.4. Imports and exports estimation (TODO)

### System

Let us consider the initial form of the system which represents the flows among non-producing countries. This can be simplified by writing everything in terms of imports, since imports and exports refer the same flows from opposite perspectives. Let us further denote the total market of a country as $M_i := C_{d,t,i}^{adj}+S_{d,t,i}^{adj}$.

$
\begin{cases} 
\sum_{k=1}^{K} I_{d,t,i\leftarrow k} = C_{d,t,i}^{adj}+S_{d,t,i}^{adj} + \sum_{l=1}^{L} E_{d,t,i\rightarrow l} \\ 
\sum_{l=1}^{L} E_{d,t,i\rightarrow l} = \sum_{l=1}^{L} I_{d,t,l\leftarrow i}
\end{cases}
\Rightarrow
\begin{cases} 
\sum_{k=1}^{K} I_{d,t,i\leftarrow k} = M_i + \sum_{l=1}^{L} I_{d,t,l\leftarrow i} \\ 
\end{cases}
$


Another complication is the effect of internal production on the balancing of flows. Therefore we must write a generalized version of the system that accurately models the overall flow of drugs among producing and non-producing countries. In order to achieve this, let us introduce the internal production $p_i$ which is $0$ for non-producing countries and may take an arbitrary positive value for producing countries.

$
\begin{cases} 
\sum_{k=1}^{K} I_{d,t,i\leftarrow k} = M_i - p_i + \sum_{l=1}^{L} I_{d,t,l\leftarrow i} \\ 
\end{cases}
$

Let us further note that given the relative weights $w_{il}$ of all inflows into country $l$, one may write $I_{d,t,l\leftarrow i} = w_{il} \cdot I_l$, where $I_l$ represents the total quantity imported by country $l$. This means that instead of solving for the values of all edges, we only need to solve for all nationa imports and national production values. 

$
\begin{cases}
I_i = (M_i - p_i) + \sum_{j=1}^{N} w_{ij} \cdot I_j
\end{cases}
$

Since there are (possibly) **$2N$ unknowns and only $N$ equations**, we are left with **$N$ degrees of freedom** corresponding to the production values.
Without accurate data on the values of national production, we would need to obtain production estimates. 

We choose to simplify the problem by considering only the biggest producers of cocaine (Bolivia, Colombia, Ecuador, and Peru) and heroin (Afghanistan, Colombia, Laos, and Myanmar) and ignoring the internal production of all other countries. 

In [5]:
# TODO

### 7.5. Additional features

We are now interated in including country-level features related to drug trafficking. These include:
* Drug prices
* GDP per capita
* Geographical coordinates
* Control of corruption
* Government effectiveness
* Stability and lack of terrorism
* Regulatory quality
* Rule of law

By combining these node-level features with the data about seizures, consumption, and markets, as well as the overall strucuture of the network (edges), we can create comprehensive network datasets. For analysis purposes, we implement both yearly and aggregate levels, as well as different formats for simple integration with PyTorch Geometric (pyg) and R. To achieve this, we make use of the *Features* module and the fuction *get_network_data* in particular.

In [20]:
import importlib
importlib.reload(Features)

cocaine_network_datasets = Features.get_network_data('Cocaine', df_ids)
heroin_network_datasets = Features.get_network_data('Heroin', df_ids)

Let us investigate the output:

In [29]:
print(f"Constructed {len(cocaine_network_datasets['pyg'])} pyg datasets for cocaine in nx.DiGraph format saved to file in .gml format;")
print(f"Constructed {len(cocaine_network_datasets['R'])} R datasets (nodes and edges) for cocaine saved to file in .csv format;")
print(f"Constructed {len(heroin_network_datasets['pyg'])} pyg datasets for heroin in nx.DiGraph format saved to file in .gml format;")
print(f"Constructed {len(heroin_network_datasets['R'])} R datasets (nodes and edges) for heroin saved to file in .csv format.")

Constructed 13 pyg datasets for cocaine in nx.DiGraph format saved to file in .gml format;
Constructed 26 R datasets (nodes and edges) for cocaine saved to file in .csv format;
Constructed 13 pyg datasets for heroin in nx.DiGraph format saved to file in .gml format;
Constructed 26 R datasets (nodes and edges) for heroin saved to file in .csv format.


## 8. Conclusion

We have undertaken a rigurous data gathering, cleaning, prcocessing, and aggregation process. In the end, we obtained **26 network datasets** corresponding to yearly and aggregate drug trafficking networks for both cocaine and heroin over a period of 12 years. Each dataset contains a different numer of nodes (countries) and edges (trafficking connections) derived from law enforcement seizures. This is by no means painting a complete picture of the global illicit trade of drugs, but it acts as a reasonable structure for gaining valuable insights.

Each node has several features relating to quantities of drugs consumed and seized, socio-economic indicators and geographical coordinates. This features will be used within our models for link prediction and network distribution. More features can be introduced based on pre-existing theories and empirical evidence from the literature.

Furthermore, each dataset has been stored in such a way as to enable data analysis and model-building with both PyTorch and PyTorch Geometric, as well as R. This will allow us to use both graph neural networks (GNN's), as well as statistical models for network analysis (e.g., exponential random graph models).

## 9. Bibliography

* Giulia Berlusconi, Alberto Aziani, Luca Giommoni, The determinants of heroin flows in Europe: A latent space approach, Social Networks, Volume 51, 2017, Pages 104-117, ISSN 0378-8733, https://doi.org/10.1016/j.socnet.2017.03.012.
(https://www.sciencedirect.com/science/article/pii/S0378873316302192)
* Alberto Aziani, Giulia Berlusconi & Luca Giommoni (2021) A Quantitative Application of Enterprise and Social Embeddedness Theories to the Transnational Trafficking of Cocaine in Europe, Deviant Behavior, 42:2, 245-267, https://doi.org/10.1080/01639625.2019.1666606. (https://www.tandfonline.com/doi/full/10.1080/01639625.2019.1666606)
* Giommoni, Luca & Berlusconi, Giulia & Aziani, Alberto. (2021). Interdicting International Drug Trafficking: a Network Approach for Coordinated and Targeted Interventions. European Journal on Criminal Policy and Research. 28.  https://doi.org/10.1007/s10610-020-09473-0 (https://link.springer.com/article/10.1007/s10610-020-09473-0#citeas)
* UNODC. (2010). Methodology–World drug report 2010. Vienna: UNODC https://www.unodc. org/documents/data-and-analysis/WDR2010/WDR2010methodology.pdf
* UNODC. (2019b). World drug report 2019 methodology report. Vienna: United Nations Office on Drugs and Crime https://wdr.unodc.org/wdr2019/prelaunch/WDR-2019-Methodology-FINAL.pdf